# Create KNN Noise

In [1]:
from baseline import ModelBase, DataBase
import pandas as pd

# Load model
model_type = 'resnet18'
number_classes = 10
device = 'cuda'
path_checkpoint = 'checkpoints/cifar10/SEED0_cifar10_resnet18_train_clean/best.pt'
model_base = ModelBase(model_type, number_classes, device)
model_base.load_model(path_checkpoint)

# Load dataloader of clean train
df_train = 'data/cifar10/processed/train.csv'
type_data = 'cifar10'
df_train = pd.read_csv(df_train)

data_base = DataBase(type_data)
dataloader = data_base.get_dataloader(
    df=df_train,
    batch_size=1,
    mode='test',
    num_workers=0
)

/media/SSD/ThangND34_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = model_base.model

In [3]:
model.eval()

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [5]:
import tqdm
features = []

for i, (img, lab) in tqdm.tqdm(enumerate(dataloader)):
    img = img.to(device)
    lab = lab.to(device)
    f = model.module.get_feature(img)
    f = f.cpu().detach().numpy().reshape(-1) # (84, )
    features.append(f)
        
features[:2]
    

0it [00:00, ?it/s]


AttributeError: 'ResNet' object has no attribute 'get_feature'

In [5]:
import random
import numpy as np

number_change = 80
indexes_change = []

while len(indexes_change) < number_change:
    ind = random.choice(np.arange(0, len(df_train)))
    if ind not in indexes_change:
        indexes_change.append(ind)

In [6]:
print(len(indexes_change), len(set(indexes_change)))


80 80


In [7]:
df_train

,path,label
0,data/cifar10/train/automobile/2496.png,1
1,data/cifar10/train/bird/1367.png,2
2,data/cifar10/train/airplane/2769.png,0
3,data/cifar10/train/deer/0475.png,4
4,data/cifar10/train/cat/3226.png,3
...,...,...
39995,data/cifar10/train/truck/4277.png,9
39996,data/cifar10/train/truck/4029.png,9
39997,data/cifar10/train/horse/2287.png,7
39998,data/cifar10/train/airplane/3214.png,0


In [8]:
df_train['isFlipped'] = 0
df_train['originLabel'] = df_train['label']
df_train

,path,label,isFlipped,originLabel
0,data/cifar10/train/automobile/2496.png,1,0,1
1,data/cifar10/train/bird/1367.png,2,0,2
2,data/cifar10/train/airplane/2769.png,0,0,0
3,data/cifar10/train/deer/0475.png,4,0,4
4,data/cifar10/train/cat/3226.png,3,0,3
...,...,...,...,...
39995,data/cifar10/train/truck/4277.png,9,0,9
39996,data/cifar10/train/truck/4029.png,9,0,9
39997,data/cifar10/train/horse/2287.png,7,0,7
39998,data/cifar10/train/airplane/3214.png,0,0,0


In [9]:
index_noise = []

sim_func_dict = {
    "dot": lambda x, y: np.dot(x, y),
    "cos": lambda x, y: distance.cosine(x, y),
    "euc": lambda x, y: np.linalg.norm(x-y)
}

# duyet moi sample
for i in tqdm.tqdm(indexes_change):
    # Tim cac NN
    nn = []
    for j, data in enumerate(dataloader):
        s = sim_func_dict["euc"](features[i], features[j])
        nn.append(s)
    nn = np.array(nn)
    idx_sorted = np.argsort(nn) # lay ra chi so tu be den lon
    # chon nhung sample gan nhat va khac nhan va chua bi dao nhan
    cnt = 0
    for t in idx_sorted:
        if t!= i and (t not in index_noise) and df_train.iloc[t]['isFlipped'] == 0 and df_train.iloc[t]['label'] != df_train.iloc[i]['label']:
            df_train.loc[t, 'isFlipped'] = 1
            df_train.loc[t, 'label'] = df_train.iloc[i]['label']
            index_noise.append(t)
            cnt += 1
        if cnt >= 100:
            break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [33:30<00:00, 25.13s/it]


In [10]:
df_train

,path,label,isFlipped,originLabel
0,data/cifar10/train/automobile/2496.png,9,1,1
1,data/cifar10/train/bird/1367.png,7,1,2
2,data/cifar10/train/airplane/2769.png,0,0,0
3,data/cifar10/train/deer/0475.png,4,0,4
4,data/cifar10/train/cat/3226.png,3,0,3
...,...,...,...,...
39995,data/cifar10/train/truck/4277.png,9,0,9
39996,data/cifar10/train/truck/4029.png,9,0,9
39997,data/cifar10/train/horse/2287.png,7,0,7
39998,data/cifar10/train/airplane/3214.png,0,0,0


In [11]:
df_train[df_train['isFlipped']==1]

,path,label,isFlipped,originLabel
0,data/cifar10/train/automobile/2496.png,9,1,1
1,data/cifar10/train/bird/1367.png,7,1,2
6,data/cifar10/train/cat/4217.png,2,1,3
22,data/cifar10/train/ship/2089.png,1,1,8
26,data/cifar10/train/airplane/0873.png,2,1,0
...,...,...,...,...
39978,data/cifar10/train/dog/2027.png,3,1,5
39982,data/cifar10/train/horse/3779.png,3,1,7
39989,data/cifar10/train/airplane/2400.png,1,1,0
39990,data/cifar10/train/deer/0902.png,6,1,4


In [12]:
df_train.to_csv('data/cifar10/processed/knn_noise_cifar10net.csv', index=False)